# Computational Theory Problems

# Problem 1 — Binary Words and Bitwise Operations

This notebook implements the core 32-bit logic functions defined in the **Secure Hash Standard (FIPS PUB 180-4, § 4.1.2)**.
These functions are used in the SHA-256 algorithm to combine 32-bit words with rotations, shifts, and Boolean operations.

The goal is to:
1. Build helper functions that safely handle 32-bit operations in NumPy.
2. Implement `Parity`, `Ch`, `Maj`, `Σ₀`, `Σ₁`, `σ₀`, and `σ₁`.
3. Document and explain each step clearly.


### Step 1 – Secure 32-bit Helpers
We use NumPy’s `uint32` type to enforce 32-bit behavior.
The helper functions `_to_u32`, `_rotr`, and `_shr` guarantee logical rotation and right-shift operations.


In [10]:
import numpy as np
from typing import Union, List

Word = np.uint32

# Force value to 32-bit unsigned word
def _to_u32(x: Union[int, np.integer]) -> Word:
    return Word(int(x) & 0xFFFFFFFF)

# Rotate-right 32-bit x by n
def _rotr(x: Word, n: int) -> Word:
    x = _to_u32(x); n = int(n) % 32
    if n == 0:
        return x
    return _to_u32((x >> n) | (x << Word(32 - n)))

# Logical right shift 32-bit x by n (zero-fill)
def _shr(x: Word, n: int) -> Word:
    x = _to_u32(x); n = int(n) % 32
    return _to_u32(x >> n)

### Step 2 – Boolean Functions
These operations mix three 32-bit words using logical bit rules:
- **Parity(x,y,z)** = x ⊕ y ⊕ z  
- **Ch(x,y,z)** = (x ∧ y) ⊕ (¬x ∧ z)  
- **Maj(x,y,z)** = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)


In [11]:
# Parity(x,y,z): XOR from 32-bit ( SHA-1 parity)
def Parity(x, y, z) -> Word:
    return _to_u32(_to_u32(x) ^ _to_u32(y) ^ _to_u32(z))

# Ch(x,y,z): (x & y) ^ (~x & z) — pick bit from y if bit x =1, otherwise z
def Ch(x, y, z) -> Word:
    x, y, z = _to_u32(x), _to_u32(y), _to_u32(z)
    return _to_u32((x & y) ^ ((~x) & z))

# Maj (Eq. 4.3): (x & y) ^ (x & z) ^ (y & z) — majority base on each bit
def Maj(x, y, z) -> Word:
    x, y, z = _to_u32(x), _to_u32(y), _to_u32(z)
    return _to_u32((x & y) ^ (x & z) ^ (y & z))


### Step 3 – Σ and σ Functions
These are rotation/shift-based transformations defined in FIPS 180-4:
- **Σ₀(x)** = ROTR² ⊕ ROTR¹³ ⊕ ROTR²²  
- **Σ₁(x)** = ROTR⁶ ⊕ ROTR¹¹ ⊕ ROTR²⁵  
- **σ₀(x)** = ROTR⁷ ⊕ ROTR¹⁸ ⊕ SHR³  
- **σ₁(x)** = ROTR¹⁷ ⊕ ROTR¹⁹ ⊕ SHR¹⁰


In [12]:
# Σ₀: ROTR² ⊕ ROTR¹³ ⊕ ROTR²²
def Sigma0(x) -> Word:
    x = _to_u32(x)
    return _to_u32(_rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22))

# Σ₁: ROTR⁶ ⊕ ROTR¹¹ ⊕ ROTR²⁵
def Sigma1(x) -> Word:
    x = _to_u32(x)
    return _to_u32(_rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25))

# σ₀: ROTR⁷ ⊕ ROTR¹⁸ ⊕ SHR³
def sigma0(x) -> Word:
    x = _to_u32(x)
    return _to_u32(_rotr(x, 7) ^ _rotr(x, 18) ^ _shr(x, 3))

# σ₁: ROTR¹⁷ ⊕ ROTR¹⁹ ⊕ SHR¹⁰
def sigma1(x) -> Word:
    x = _to_u32(x)
    return _to_u32(_rotr(x, 17) ^ _rotr(x, 19) ^ _shr(x, 10))


### Step 4 – Demonstration with Fixed Values
We test the functions using example 32-bit words from the SHA-256 initial values.


In [13]:
# Predefined 32-bit demo inputs
x = np.uint32(0x6a09e667)
y = np.uint32(0x12345678)
z = np.uint32(0xdeadbeef)

print("===== INPUT VALUES =====")
print(f"x = {hex(int(x))}")
print(f"y = {hex(int(y))}")
print(f"z = {hex(int(z))}")

print("\n===== LOGIC FUNCTIONS =====")
print(f"Parity(x, y, z) = {hex(int(Parity(x, y, z)))}")
print(f"Ch(x, y, z)     = {hex(int(Ch(x, y, z)))}")
print(f"Maj(x, y, z)    = {hex(int(Maj(x, y, z)))}")

print("\n===== SIGMA FUNCTIONS =====")
print(f"Sigma0(x) = {hex(int(Sigma0(x)))}")
print(f"Sigma1(x) = {hex(int(Sigma1(x)))}")
print(f"sigma0(x) = {hex(int(sigma0(x)))}")
print(f"sigma1(x) = {hex(int(sigma1(x)))}")


===== INPUT VALUES =====
x = 0x6a09e667
y = 0x12345678
z = 0xdeadbeef

===== LOGIC FUNCTIONS =====
Parity(x, y, z) = 0xa6900ef0
Ch(x, y, z)     = 0x96a45ee8
Maj(x, y, z)    = 0x5a2df66f

===== SIGMA FUNCTIONS =====
Sigma0(x) = 0xce20b47e
Sigma1(x) = 0x55b65510
sigma0(x) = 0xba0cf582
sigma1(x) = 0xcfe5da3c


### Step 5 – Reflection and Research Discussion
According to **FIPS PUB 180-4** (NIST, 2015), these functions form the
non-linear mixing stage of SHA-256.  
Each rotation and shift ensures diffusion and bit independence.

Sources:
- [NIST FIPS 180-4 (2015) — Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)
- Numpy documentation on [Unsigned integer types](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32)


# Problem 2 — Fractional Parts of Cube Roots

The goal of this problem is to compute the **64 constants K₀ – K₆₃** used in the **SHA-256** hashing algorithm, as defined in  
**FIPS PUB 180-4 (§ 4.2.2 – SHA-224 and SHA-256 Constants)**.  
Each constant is obtained by taking the **first 32 bits of the fractional part** of the cube root of the first 64 prime numbers.

These constants are fundamental for the *message schedule* and *compression function* of SHA-256, ensuring strong diffusion and independence between rounds.




### Step 1: Generate the first 64 prime numbers

In [14]:

# Generate the first `n` prime numbers using a dynamic sieve of Eratosthenes.
def primes(n: int) -> np.ndarray:
 
    # If user asks for less than 1 prime, return an empty array
    if n < 1:
        return np.array([], dtype=int)

    # Estimate an upper bound for the nth prime.
    # For small n, we just pick a small constant.
    # For larger n, we use the prime number theorem approximation:
    #   nth prime ≈ n * (log n + log log n)
    if n < 6:
        bound = 15
    else:
        nf = float(n)
        bound = int(nf * (np.log(nf) + np.log(np.log(nf))) + 50)  # add small buffer

    # Inner helper function: sieve of Eratosthenes up to `limit`
    def sieve(limit: int) -> List[int]:
        # Create boolean array representing numbers 0..limit (True = prime)
        arr = np.ones(limit + 1, dtype=bool)
        arr[:2] = False  # 0 and 1 are not prime

        # Eliminate non-prime numbers by marking multiples as False
        for p in range(2, int(limit**0.5) + 1):
            if arr[p]:
                arr[p*p:limit+1:p] = False  # mark all multiples of p

        # Return list of primes found
        return np.flatnonzero(arr).tolist()

    # Generate primes up to the current bound
    ps = sieve(bound)

    # If not enough primes, double the bound and try again until we have n primes
    while len(ps) < n:
        bound *= 2
        ps = sieve(bound)

    # Return only the first n primes as numpy array
    return np.array(ps[:n], dtype=int)


### Step 2: Cube roots of the first 64 primes


In [15]:
def cube_root_constants(n: int = 64) -> np.ndarray:
    # Generate n primes. 
    p = primes(n).astype(np.float64)
    # Compute cube roots using NumPy (float64 precision).
    roots = np.cbrt(p)    
    # Take fractional part (root − floor(root)).                       
    frac  = roots - np.floor(roots)
    # Multiply by 2^32 and floor to integer.            
    scaled = np.floor(frac * (2**32))
    # Cast to np.uint32 to enforce 32-bit size.         
    return scaled.astype(np.uint32)

### Step 3: Display results in hex and verify

In [16]:
def display_constants(k_values: np.ndarray) -> None:

    # Convert each constant in k_values to an 8-digit lowercase hexadecimal string.
    # Example: 1116352408 → "428a2f98"
    hex_vals = [f"{int(v):08x}" for v in k_values]

    # The official list of 64 constants from the SHA-256 standard (FIPS 180-4 §4.2.2)
    ref_hex = [
        "428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5",
        "d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174",
        "e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da",
        "983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967",
        "27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85",
        "a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070",
        "19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3",
        "748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2",
    ]

    # Compare each calculated constant with the official reference
    matches = [calc == ref for calc, ref in zip(hex_vals, ref_hex)]
    all_match = all(matches)  # True if all 64 match perfectly

    # Print formatted constants
    print("===== SHA-256 Constants (K[0..63]) =====")
    for i, hx in enumerate(hex_vals):
        print(f"K[{i:02}] = 0x{hx}")

    # Print verification summary
    print("\nAll 64 constants match FIPS 180-4 reference:", all_match)


### Step 4: Main execution

In [17]:
if __name__ == "__main__":
    # Step 1: Generate constants — each K[i] = fractional part of (prime_i)^(1/3) * 2^32
    # The cube_root_constants() function handles this math.
    K = cube_root_constants(64)

    # Step 2: Display and verify all constants.
    # This will print K[0..63] in hex and confirm whether they match
    # the official SHA-256 specification.
    display_constants(K)


===== SHA-256 Constants (K[0..63]) =====
K[00] = 0x428a2f98
K[01] = 0x71374491
K[02] = 0xb5c0fbcf
K[03] = 0xe9b5dba5
K[04] = 0x3956c25b
K[05] = 0x59f111f1
K[06] = 0x923f82a4
K[07] = 0xab1c5ed5
K[08] = 0xd807aa98
K[09] = 0x12835b01
K[10] = 0x243185be
K[11] = 0x550c7dc3
K[12] = 0x72be5d74
K[13] = 0x80deb1fe
K[14] = 0x9bdc06a7
K[15] = 0xc19bf174
K[16] = 0xe49b69c1
K[17] = 0xefbe4786
K[18] = 0x0fc19dc6
K[19] = 0x240ca1cc
K[20] = 0x2de92c6f
K[21] = 0x4a7484aa
K[22] = 0x5cb0a9dc
K[23] = 0x76f988da
K[24] = 0x983e5152
K[25] = 0xa831c66d
K[26] = 0xb00327c8
K[27] = 0xbf597fc7
K[28] = 0xc6e00bf3
K[29] = 0xd5a79147
K[30] = 0x06ca6351
K[31] = 0x14292967
K[32] = 0x27b70a85
K[33] = 0x2e1b2138
K[34] = 0x4d2c6dfc
K[35] = 0x53380d13
K[36] = 0x650a7354
K[37] = 0x766a0abb
K[38] = 0x81c2c92e
K[39] = 0x92722c85
K[40] = 0xa2bfe8a1
K[41] = 0xa81a664b
K[42] = 0xc24b8b70
K[43] = 0xc76c51a3
K[44] = 0xd192e819
K[45] = 0xd6990624
K[46] = 0xf40e3585
K[47] = 0x106aa070
K[48] = 0x19a4c116
K[49] = 0x1e376c08
K[50] = 0


### References

- **NIST FIPS PUB 180-4 (2015)** – *Secure Hash Standard (SHS)*.  
  [https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)  
- **NumPy Documentation** – [Unsigned integer types (`numpy.uint32`)](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32)  
- Rosser, J.B., & Schoenfeld, L. (1962). Approximate formulas for some functions of prime numbers.  
- FIPS Annex A – Table of Constants for SHA-224 and SHA-256.  


## Problem 3: Padding

### Core Requirements
1. Function Type: Write a Python generator function (using yield).
2. Input: The function accepts a bytes object named msg.
3. Standard Compliance: The function must process the message according to Section 5.1.1 (Padding the Message) and Section 5.2.1 (Parsing the Message) of the Secure Hash Standard (FIPS PUB 180-4). These sections apply to SHA-1, SHA-224, and SHA-256.
4. Block Size: For these algorithms, the block size is 512 bits.
5. Output: At each iteration, the generator should yield the next 512-bit block of the padded message as a bytes object

#### Yield 512-bit (64-byte) blocks of `msg` with SHA-1/SHA-224/SHA-256 padding, per FIPS 180-4 §5.1.1 (Padding the Message) and §5.2.1 (Parsing the Message).

    Padding rules (for 512-bit block algorithms):
      1) append a single '1' bit (0x80),
      2) append k '0' bits so total length ≡ 448 (mod 512),
      3) append 64-bit big-endian integer = original message length in bits.

    The generator yields each 64-byte block as `bytes`.

In [18]:
def block_parse(msg: bytes):
    ml_bytes = len(msg)
    ml_bits = ml_bytes * 8
    print(f"Original message length: {ml_bytes} bytes ({ml_bits} bits)")

    # Yield all full 512-bit blocks from the message first
    i = 0
    while i + 64 <= ml_bytes:
        block = msg[i:i+64]
        print(f"Block {i//64 + 1}: {block.hex()}")
        yield block
        i += 64

    # ---- Padding phase ----
    tail = msg[i:] + b'\x80'                       # Step 1: append '1' bit (10000000)
    pad_zeros = (56 - (len(tail) % 64)) % 64       # Step 2: pad zeros until ≡ 56 mod 64
    tail += b'\x00' * pad_zeros
    tail += ml_bits.to_bytes(8, 'big')             # Step 3: append 64-bit message length

    print(f"After padding: total {len(tail)} bytes, "
          f"last 8 bytes = {ml_bits.to_bytes(8, 'big').hex()}")

    # ---- Yield final padded blocks ----
    for j in range(0, len(tail), 64):
        block = tail[j:j+64]
        print(f"Padded Block {i//64 + (j//64) + 1}: {block.hex()}")
        yield block


In [19]:
def hex_blocks(blocks):
    return [b.hex() for b in blocks]

# 0) Empty message
blocks = list(block_parse(b""))
assert len(blocks) == 1
# First byte 0x80, last 8 bytes = 0x...0000 (length=0)
assert blocks[0][0] == 0x80 and blocks[0][-8:] == (0).to_bytes(8, 'big')

# 1) "abc" (24 bits)
blocks = list(block_parse(b"abc"))
assert len(blocks) == 1
assert blocks[0][-8:] == (24).to_bytes(8, 'big')  # 0x...0018

# 2) 56-byte input → must produce TWO blocks (because 0x80 won’t fit with length in the same block)
blocks = list(block_parse(b"A"*56))
assert len(blocks) == 2
# Last 8 bytes of final block = 56*8 bits
assert blocks[-1][-8:] == (56*8).to_bytes(8, 'big')

# 3) 64-byte input → also two blocks (first is raw 64, second is pure padding+length)
blocks = list(block_parse(b"B"*64))
assert len(blocks) == 2
assert blocks[-1][-8:] == (64*8).to_bytes(8, 'big')



Original message length: 0 bytes (0 bits)
After padding: total 64 bytes, last 8 bytes = 0000000000000000
Padded Block 1: 80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Original message length: 3 bytes (24 bits)
After padding: total 64 bytes, last 8 bytes = 0000000000000018
Padded Block 1: 61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
Original message length: 56 bytes (448 bits)
After padding: total 128 bytes, last 8 bytes = 00000000000001c0
Padded Block 1: 41414141414141414141414141414141414141414141414141414141414141414141414141414141414141414141414141414141414141418000000000000000
Padded Block 2: 000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001c0
Original message length: 64 bytes (512 bits)
Block 1: 4242424242424242424242424242424242424242424242

## Problem 4: Hashes

### Introduction
Problem 4 implements Section 6.2.2 SHA-256 Hash Computation from the Secure Hash Standard (FIPS PUB 180-4). This is the core algorithm that computes the next intermediate hash value from the current hash value and a 512-bit message block.

### Objective
Write a function hash(current, block) that accepts:<br>
current: Current hash value H^(i-1) (8 32-bit words)<br>
block: Next message block M^(i) (512 bits = 64 bytes)<br>
Returns: Next hash value H^(i) (8 32-bit words)

In [20]:
import numpy as np


#### Step 1: Prepare Message Schedule {W_t}
The message schedule is an array of 64 32-bit words (W_0 through W_63) generated from the message block.<br>
#### Theory
First 16 words (t = 0..15): Taken directly from the message block<br>
W_t = M^(i)_t  for 0 ≤ t ≤ 15<br>
Remaining 48 words (t = 16..63): Computed recursively<br>
W_t = σ₁(W_{t-2}) + W_{t-7} + σ₀(W_{t-15}) + W_{t-16}<br>
(All additions modulo 2^32)

#### Code Implementation

In [21]:
def prepare_message_schedule(block):
    """
    Prepare the message schedule {W_t} from a 512-bit block.
    
    Parameters:
        block (bytes): 64-byte message block (512 bits)
    
    Returns:
        numpy.ndarray: Array of 64 32-bit words
    """
    import numpy as np
    
    # Initialize array of 64 words
    W = np.zeros(64, dtype=np.uint32)
    
    # First 16 words: split block into 16 32-bit words (big-endian)
    for t in range(16):
        W[t] = int.from_bytes(block[t*4:(t+1)*4], byteorder='big')
    
    # Remaining 48 words: computed using the formula
    for t in range(16, 64):
        W[t] = (sigma1(W[t-2]) + W[t-7] + 
                sigma0(W[t-15]) + W[t-16]) & 0xFFFFFFFF
    
    return W

### Step 2: Initialize Working Variables
Eight working variables (a, b, c, d, e, f, g, h) are initialized from the current hash value.

In [22]:
def initialize_working_variables(current):
    """
    Initialize 8 working variables from the current hash value.
    
    Parameters:
        current (list/array): 8 32-bit words of H^(i-1)
    
    Returns:
        tuple: (a, b, c, d, e, f, g, h)
    """
    import numpy as np
    a, b, c, d, e, f, g, h = [np.uint32(x) for x in current]
    return a, b, c, d, e, f, g, h

### Step 3: Main Loop (64 Rounds)
This is the most critical part - 64 rounds of transformations on the working variables.
#### Theory
For each round t (0 ≤ t ≤ 63):<br>
Compute T₁: T₁ = h + Σ₁(e) + Ch(e,f,g) + K_t + W_t<br>
Compute T₂: T₂ = Σ₀(a) + Maj(a,b,c)<br>
Update variables:

h = g<br>
   g = f<br>
   f = e<br>
   e = d + T₁<br>
   d = c<br>
   c = b<br>
   b = a<br>
   a = T₁ + T₂<br>

#### Code Implementation

In [23]:
def compression_function(current, W, K):
    """
    Execute 64 rounds of the SHA-256 compression function.
    
    Parameters:
        current: Current hash value (8 32-bit words)
        W: Message schedule (64 32-bit words)
        K: Array of constants K (64 32-bit words)
    
    Returns:
        tuple: Final 8 working variables (a,b,c,d,e,f,g,h)
    """
    import numpy as np
    
    # Initialize working variables
    a, b, c, d, e, f, g, h = initialize_working_variables(current)
    
    # 64 rounds
    for t in range(64):
        # Compute T1
        T1 = (h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t]) & 0xFFFFFFFF
        
        # Compute T2
        T2 = (Sigma0(a) + Maj(a, b, c)) & 0xFFFFFFFF
        
        # Update variables (shift and add T1, T2)
        h = g
        g = f
        f = e
        e = (d + T1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (T1 + T2) & 0xFFFFFFFF
    
    return a, b, c, d, e, f, g, h

### Step 4: Compute New Hash Value H^(i)
After 64 rounds, add the working variables to the old hash value.

In [24]:
def compute_intermediate_hash(current, working_vars):
    """
    Compute the intermediate hash value H^(i).
    
    Parameters:
        current: Hash value H^(i-1) (8 32-bit words)
        working_vars: Tuple (a,b,c,d,e,f,g,h) after 64 rounds
    
    Returns:
        numpy.ndarray: Hash value H^(i) (8 32-bit words)
    """
    import numpy as np
    
    a, b, c, d, e, f, g, h = working_vars
    H = np.zeros(8, dtype=np.uint32)
    
    # H^(i)_j = H^(i-1)_j + variable_j (mod 2^32)
    H[0] = (current[0] + a) & 0xFFFFFFFF
    H[1] = (current[1] + b) & 0xFFFFFFFF
    H[2] = (current[2] + c) & 0xFFFFFFFF
    H[3] = (current[3] + d) & 0xFFFFFFFF
    H[4] = (current[4] + e) & 0xFFFFFFFF
    H[5] = (current[5] + f) & 0xFFFFFFFF
    H[6] = (current[6] + g) & 0xFFFFFFFF
    H[7] = (current[7] + h) & 0xFFFFFFFF
    
    return H

### Complete hash(current, block) Function
Combining all steps above:

In [25]:
def hash(current, block):
    """
    Compute the next SHA-256 hash value according to Section 6.2.2.
    
    This function implements the core SHA-256 compression function,
    which processes a single 512-bit block and updates the hash value.
    
    Parameters:
        current (array-like): Current hash value H^(i-1) (8 32-bit words)
        block (bytes): Message block M^(i) (64 bytes = 512 bits)
    
    Returns:
        numpy.ndarray: Next hash value H^(i) (8 32-bit words)
    
    References:
        FIPS PUB 180-4, Section 6.2.2: SHA-256 Hash Computation
        https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    """
    import numpy as np
    
    # Ensure current is a numpy array
    current = np.array(current, dtype=np.uint32)
    
    # Step 1: Prepare message schedule
    W = prepare_message_schedule(block)
    
    # Steps 2 & 3: Compression function (64 rounds)
    working_vars = compression_function(current, W, K)
    
    # Step 4: Compute new hash value
    H = compute_intermediate_hash(current, working_vars)
    
    return H

### Testing
Test with Standard Vector

In [26]:
# Initial hash value H^(0) from Section 5.3.3
H0 = np.array([
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
], dtype=np.uint32)

# Test block (example: first block of "abc" with padding)
# "abc" = 0x616263, followed by padding
test_block = b'abc' + b'\x80' + b'\x00' * 55 + b'\x00\x00\x00\x00\x00\x00\x00\x18'

# Compute hash
H1 = hash(H0, test_block)

print("H^(1):", ' '.join(f'{h:08x}' for h in H1))

# Expected result for "abc":
# ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad

H^(1): 5b2beac7 edfc3d10 5f66435f 0ddf6c8b a1a07ff7 84229bf0 7ac92a88 eb4756ec


/tmp/ipykernel_2798/1845074668.py:22: RuntimeWarning: overflow encountered in scalar add
  W[t] = (sigma1(W[t-2]) + W[t-7] +
/tmp/ipykernel_2798/3338691485.py:21: RuntimeWarning: overflow encountered in scalar add
  T1 = (h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t]) & 0xFFFFFFFF
/tmp/ipykernel_2798/3338691485.py:24: RuntimeWarning: overflow encountered in scalar add
  T2 = (Sigma0(a) + Maj(a, b, c)) & 0xFFFFFFFF
/tmp/ipykernel_2798/3338691485.py:30: RuntimeWarning: overflow encountered in scalar add
  e = (d + T1) & 0xFFFFFFFF
/tmp/ipykernel_2798/3338691485.py:34: RuntimeWarning: overflow encountered in scalar add
  a = (T1 + T2) & 0xFFFFFFFF
/tmp/ipykernel_2798/681885099.py:18: RuntimeWarning: overflow encountered in scalar add
  H[0] = (current[0] + a) & 0xFFFFFFFF
/tmp/ipykernel_2798/681885099.py:23: RuntimeWarning: overflow encountered in scalar add
  H[5] = (current[5] + f) & 0xFFFFFFFF


## Problem 5: Passwords

In [ ]:
1

1